### 【 손글씨 숫자 인식 모델 구현 】
- 데이터셋 : mnist_train.csv, mnist_test.csv
- 학습종류 : 자동학습 - 다중클래스분류
- 학습방법 : 인공신경망기반

[1] 모듈 로딩 및 데이터 준비 <hr>

In [153]:
# [1-1] 모듈 로딩
import pandas as pd

import torch                                        # 텐서 및 수치과학 함수들 관련 모듈
import torch.nn as nn                               # 신경망 층 관련 모듈
import torch.nn.functional as F                     # 신경망 함수들(AF, LF, MF) 모듈
import torch.optim as optim                         # 신경망 최적화 모듈
from torch.optim import Adam                        # Adam 최적화 알고리즘

from torch.utils.data import Dataset, DataLoader    # pytorch의 데이터 로딩
from torch.utils.data import Subset                 # pytorch의 데이터셋 관련 모듈
from sklearn.model_selection import train_test_split

from torchinfo import summary

In [154]:
# [1-2] 데이터 준비
TRAIN_FILE = '../Data/mnist_train.csv'
TEST_FILE  = '../Data/mnist_test.csv'

# [1-3] 데이터 로딩
trainDF = pd.read_csv(TRAIN_FILE, header=None)
testDF  = pd.read_csv(TEST_FILE, header=None)

[2] 커스텀 데이터셋 준비 <hr>

In [155]:
# -------------------------------------------------------------------------------------
# [2-1] 커스텀 데이터셋 클래스 정의
# -------------------------------------------------------------------------------------
# 클래스이름 : ClfDataset
# 부모클래스 : Dataset
# 오버라이딩 : _ _init_ _(self)         : [필수] 피쳐, 타겟, [선택]행수, 컬럼수, 타겟 수...
#            _ _len_ _(self)          : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
#            _ _getitem_ _(self, idx) : 인스턴스명[idx] 시 자동 호출,
#                                       idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
# -------------------------------------------------------------------------------------
class ClfDataset(Dataset):

    #- 피쳐와 타겟 저장 및 기타 속성 초기화
    def __init__(self, dataDF):
        super().__init__()
        ## 피쳐, 타겟 초기화 필수
        self.x = dataDF[dataDF.columns[1:]].values
        self.y = dataDF[dataDF.columns[0]].values


    #- 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.x.shape[0]
    
    #- 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[index]에 자동호출됨
    def __getitem__(self, index):
        xTS = torch.tensor(self.x[index], dtype=torch.float32)
        yTS = torch.tensor(self.y[index], dtype=torch.long)
        return xTS, yTS

In [156]:
# -------------------------------------------------------------------------------------
# [2-2] 커스텀 데이터셋 인스턴스 생성 및 사용
# -------------------------------------------------------------------------------------
allDS   = ClfDataset(trainDF)   ## <= trainDS, validDS 분리
testDS  = ClfDataset(testDF)

print(f'allDS : {len(allDS)},  testDS : {len(testDS)}')

allDS : 60000,  testDS : 10000


In [157]:
# -------------------------------------------------------------------------------------
# [2-3] 학습용/검증용/테스트용 데이터셋 분리
# -------------------------------------------------------------------------------------
# 학습용   : 순수 학습에 즉, 데이터셋에 규칙/패턴을 찾기 위한 데이터셋
# 검증용   : 제대로 데이터셋에서 규칙/패턴을 찾는지 확인 용도
#           에포크 단위로 찾은 규칙/패턴의 검증용으로 사용
# 테스트용 : 데이터셋에 규칙/패턴 찾은 후 최종 테스트용으로 사용

# 학습용 데이터셋에서 타겟/라벨만 추출
targetList = allDS.y

# 학습용 데이터셋에서 타겟/라벨만 추출
dataNP = allDS.y.shape

# 학습용 데이터셋에서 타겟/라벨 인덱스 생성
dataIndexList = list(range(len(allDS)))

In [158]:
# 학습용/검증용 데이터셋 인덱스 분리
# - train_test_split() 함수 : train:test : 75:27 비율로 학습용, 테스트용 데이터셋 분리
#                            stratify : 분류용 데이터셋의 경우 카테고리 비율 유지해서 데이터셋 분리
X_trainIdx, X_validIdx, y_train, y_valid = train_test_split(dataIndexList,
                                                            targetList,
                                                            train_size=0.8,
                                                            stratify=targetList,
                                                            random_state=10)

# print(f"X_trainIdx : {len(X_trainIdx)}개, {type(X_trainIdx)}")
# print(f"X_validIdx : {len(X_validIdx)}개, {type(X_validIdx)}")
# print(f"testDS     : {len(testDS)}개")

In [159]:
# -------------------------------------------------------------------------------------
# 학습용/검증용 데이터셋 생성 ----> Dataset ---> 2개 Subset 분리
# -------------------------------------------------------------------------------------
trainDS = Subset(allDS, X_trainIdx)
validDS = Subset(allDS, X_validIdx)

print(f"allDS   : {len(allDS)}개, {type(allDS)}")
print(f"trainDS : {len(trainDS)}개, {type(trainDS)}")
print(f"validDS : {len(validDS)}개, {type(validDS)}")

allDS   : 60000개, <class '__main__.ClfDataset'>
trainDS : 48000개, <class 'torch.utils.data.dataset.Subset'>
validDS : 12000개, <class 'torch.utils.data.dataset.Subset'>


[3] 모델 클래스 설계 <HR>

In [160]:
# -------------------------------------------------------------------------------------
# [3-1] 커스텀 모델 클래스 정의
# -------------------------------------------------------------------------------------
# 클래스이름 : MNISTModel
# 부모클래스 : nn.Module
# 오버라이딩 : __init__(self)
#            forward(self, x)
# 
# 데이터셋:
# - allDS: 60000개 (전체 MNIST 훈련 데이터)
# - trainDS: 48000개 (80% 분할)
# - validDS: 12000개 (20% 분할)
# - 다중클래스 분류: 0~9 (10개 클래스)
# -------------------------------------------------------------------------------------
#               입력수           퍼셉트론수/출력수           AF(활성화함수)
# -------------------------------------------------------------------------------------
# 입력층           784                   784                
# 은닉층1          784                   256                    ReLU
# 은닉층2          256                   128                    ReLU
# 은닉층3          128                    64                    ReLU
# 출력층           64                     10                   Softmax  다중분류
# -------------------------------------------------------------------------------------
class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 입력: 784 (28x28 flatten)
        # 은닉층1: 256 노드
        # 은닉층2: 128 노드
        # 은닉층3: 64 노드
        # 출력: 10 (0~9 숫자 분류)
        
        self.hd1_layer = nn.Linear(784, 256)      # 첫 번째 완전연결층
        self.hd2_layer = nn.Linear(256, 128)      # 두 번째 완전연결층
        self.hd3_layer = nn.Linear(128, 64)       # 세 번째 완전연결층
        self.out_layer = nn.Linear(64, 10)        # 출력층
        
        self.relu = nn.ReLU()               # 활성화 함수
        
    def forward(self, x):
        
        out = F.relu(self.hd1_layer(x))
        out = F.relu(self.hd2_layer(out))
        out = F.relu(self.hd3_layer(out))
        out = self.out_layer(out) # CrossEntropyLoss 안에 Softmax 내장
        
        return out

In [161]:
# -------------------------------------------------------------------------------------
# [3-2] 커스텀 모델 클래스 구조 확인 -> torchinfo 활용
# -------------------------------------------------------------------------------------
# 모델 인스턴스 생성
model = MNISTModel()
print(model)

# 모델 구조 출력
summary(model, input_size=(1, 784))

MNISTModel(
  (hd1_layer): Linear(in_features=784, out_features=256, bias=True)
  (hd2_layer): Linear(in_features=256, out_features=128, bias=True)
  (hd3_layer): Linear(in_features=128, out_features=64, bias=True)
  (out_layer): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)


Layer (type:depth-idx)                   Output Shape              Param #
MNISTModel                               [1, 10]                   --
├─Linear: 1-1                            [1, 256]                  200,960
├─Linear: 1-2                            [1, 128]                  32,896
├─Linear: 1-3                            [1, 64]                   8,256
├─Linear: 1-4                            [1, 10]                   650
Total params: 242,762
Trainable params: 242,762
Non-trainable params: 0
Total mult-adds (M): 0.24
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.97
Estimated Total Size (MB): 0.98

[4] 학습 준비 <hr>

In [162]:
# -------------------------------------------------------------------------------------
# [4-1] 학습 설정
# -------------------------------------------------------------------------------------
# 학습 진행 횟수 및 학습량, 학습 진행 위치, W/b 업데이트 간격
EPOCHS     = 1
BATCH_SIZE = 200
LR         = 0.01
DEVICE     = 'cuda' if torch.cuda.is_available() else 'cpu'

# -------------------------------------------------------------------------------------
# [4-2] 학습 인스턴스
# -------------------------------------------------------------------------------------
# Model 인스턴스 : 자동으로 층별 W, b 텐서 생성 및 랜덤 초기화
loss_fn = nn.CrossEntropyLoss()  # softmax + log + NLLLoss 자동 적용
                                  
# 손실함수 인스턴스 : 다중분류용
lossFn = nn.CrossEntropyLoss()

# 최적화 인스턴스 : 모델의 층별 파라미터 즉, W, b 업데이트
optim = optim.Adam(model.parameters(), lr=LR)

# 데이터로더 인스턴스 : 학습/검증/테스트에 사용될 학습량만큼 데이터 추출
trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE, shuffle=True)
validDL = DataLoader(validDS, batch_size = BATCH_SIZE)
testDL = DataLoader(testDS, batch_size = BATCH_SIZE)

[5] 학습 진행 <HR>

In [163]:
import util_func as uf

In [164]:
# ---> 학습 진행
# - 학습과 검증 결과 저장 : 학습 진행 / 중단 여부 결정, 모델 저장 여부
TRAIN_LA = {"loss" : [], "acc" : []}
VALID_LA = {"loss" : [], "acc" : []}

# - 지정된 학습 횟수 만큼 학습 진행 & 에포크 단위로 학습과 검증 결과 저장
for epoch in range(EPOCHS):
    # 1에포크 학습
    train_loss, train_acc = uf.train_one_epoch(model, trainDL, loss_fn, optim, DEVICE)

    # 1에포크 학습 후 업데이트 W, b 검사 : 학습에 사용되지 않는 데이터
    valid_loss, valid_acc = uf.evaluate(model, validDL, lossFn, DEVICE)

    # 학습과 검증 결과 저장
    TRAIN_LA["loss"].append(train_loss)
    TRAIN_LA["acc"].append(train_acc)
    VALID_LA["loss"].append(valid_loss)
    VALID_LA["acc"].append(valid_acc)

    # 진행 상황 출력
    # [EPOCH-1] TRAIN => Loss : xxx     Acc : xxx       VALID => Loss : xxx     Acc : xxx
    print(f"[EPOCH-{epoch:03}] TRAIN => Loss : {train_loss:.7f}     Acc : {train_acc:.5f}")
    print(f'{" "*12}VALID => Loss : {valid_loss:.7f}     Acc : {valid_acc:.5f}')

[EPOCH-000] TRAIN => Loss : 1.9693599     Acc : 0.75773
            VALID => Loss : 0.3944545     Acc : 0.88450
